In [11]:
import numpy as np
import os
import csv
import glob
from scipy.signal import butter, filtfilt 
import scipy.fftpack

In [12]:
preprocessed_name = "dft321"

# input_dir and out_dir
input_dir_path = "../rawdatas/*/*.csv"
input_dir = glob.glob(input_dir_path)
out_dir_path = "../preprocessed/train_"+preprocessed_name

In [13]:
fs = 1000.0 # サンプリング周波数 1000Hz => 1m秒間隔でサンプリング 

#通常用FFT
def ffta(data):
    return np.fft.fft(data)

def butter_lowpass(cutoff, fs, order=5): 
    nyq = 0.5 * fs 
    normal_cutoff = cutoff/nyq 
    b, a = butter(order, normal_cutoff, btype='low', analog=False) 
    return b, a 

def butter_lowpass_filtfilt(data, cutoff, fs, order=5): 
    b, a = butter_lowpass(cutoff, fs, order=order) 
    y = filtfilt(b, a, data) 
    return y 
  
def smtdA(csvdata):

    N = len(csvdata) # FFTのサンプル数 
    Ax = np.fft.fft(csvdata) 
    
    Ax_abs = np.abs(Ax) # 振幅スペクトル
   
    theta = np.angle(Ax)

    cutoff = 10
    condA = butter_lowpass_filtfilt(Ax_abs, cutoff, fs) #ゼロ位相遅延フィルタをかけて平滑化
    condA_real = condA*np.cos(theta)
    condA_imag = condA*np.sin(theta)
    
    return condA,condA_real,condA_imag

In [14]:
for d in input_dir:
    print(d)
    if ".DS_Store" in d:
        os.remove(d)
        input_dir.remove(d)
        continue
    
    end_index = d.rfind('/')
    os.makedirs(out_dir_path+'/'+d[12:end_index], exist_ok=True)
        
    data = np.loadtxt(d, delimiter=",")
    
    axis_time = np.vstack(data[:,0])
    axis_x = np.vstack(data[:, 1])
    axis_y = np.vstack(data[:, 2])
    axis_z = np.vstack(data[:, 3])

    print(axis_x.shape)
    #smoothing
    Ax, real_x, imag_x = smtdA(axis_x.T)
    Ay, real_y, imag_y = smtdA(axis_y.T)
    Az, real_z, imag_z = smtdA(axis_z.T)
    
    #DFT321式（３）
    A_s = np.sqrt(np.abs(Ax)**2+np.abs(Ay)**2+np.abs(Az)**2)

    #
    real_sum = real_x+real_y+real_z
    imag_sum = imag_x+imag_y+imag_z

    theta = np.angle(real_sum+1j*imag_sum)

    ifftA = np.fft.ifft(A_s * np.exp(1j * theta))
    realA = ifftA.real.T
    print(realA.shape)
    preprocessed = np.hstack([axis_time,realA])
    np.savetxt(out_dir_path+'/'+d[12:-4]+'_'+preprocessed_name+'.csv', preprocessed, delimiter=',')

../rawdatas/Luncheon01/Luncheon01_15.csv
(7524, 1)
(7524, 1)
../rawdatas/Luncheon01/Luncheon01_01.csv
(7514, 1)
(7514, 1)
../rawdatas/Luncheon01/Luncheon01_29.csv
(7781, 1)
(7781, 1)
../rawdatas/Luncheon01/Luncheon01_28.csv
(7339, 1)
(7339, 1)
../rawdatas/Luncheon01/Luncheon01_14.csv
(7446, 1)
(7446, 1)
../rawdatas/Luncheon01/Luncheon01_02.csv
(7836, 1)
(7836, 1)
../rawdatas/Luncheon01/Luncheon01_16.csv
(7224, 1)
(7224, 1)
../rawdatas/Luncheon01/Luncheon01_17.csv
(7731, 1)
(7731, 1)
../rawdatas/Luncheon01/Luncheon01_03.csv
(8102, 1)
(8102, 1)
../rawdatas/Luncheon01/Luncheon01_07.csv
(8102, 1)
(8102, 1)
../rawdatas/Luncheon01/Luncheon01_13.csv
(8081, 1)
(8081, 1)
../rawdatas/Luncheon01/Luncheon01_12.csv
(7926, 1)
(7926, 1)
../rawdatas/Luncheon01/Luncheon01_06.csv
(8428, 1)
(8428, 1)
../rawdatas/Luncheon01/Luncheon01_38.csv
(7876, 1)
(7876, 1)
../rawdatas/Luncheon01/Luncheon01_10.csv
(7665, 1)
(7665, 1)
../rawdatas/Luncheon01/Luncheon01_04.csv
(8655, 1)
(8655, 1)
../rawdatas/Luncheon01/L

NameError: name 'a' is not defined